<a href="https://colab.research.google.com/github/aakashratha1006/Improving-Training-Speed-Of-Neural-Networks/blob/master/PDC_J_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 uninstall tensorflow
!pip3 install tensorflow==2.0.0
!pip3 install grpcio==1.24.3

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 86.3MB 110kB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 450kB 38.4MB/s 
     |████████████████████████████████| 3.8MB 37.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=fefa971aa5ec6bcf574137326fb20398368723ec4a0ec19d078a467ba77b37f7
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probabil

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [ ]:
import tensorflow_datasets as tfds
import os, json
import numpy


In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
_NUM_CLASSES = 10
_MODEL_DIR = "ModelName"
_NUM_CHANNELS = 1
_IMG_SIZE = 28
_LEARNING_RATE = 0.05
_NUM_EPOCHS = 20
_BATCH_SIZE = 2048

In [ ]:
def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',with_info=True,as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,input_context.input_pipeline_id)

  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def model_fn(features, labels, mode):

  global_step=tf.compat.v1.train.get_or_create_global_step()

  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  global_step=tf.compat.v1.train.get_global_step()
    
  images = tf.reshape(features, [-1, _IMG_SIZE, _IMG_SIZE, _NUM_CHANNELS])
  
  logits = model(images)
  predicted_logit = tf.argmax(input=logits, axis=1, output_type=tf.int32)
  probabilities = tf.nn.softmax(logits)
    
    


    
  #PREDICT
  predictions = {
    "predicted_logit": predicted_logit,
    "probabilities": probabilities
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)




  with tf.name_scope('loss'):
    cross_entropy = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, scope='loss')
    tf.summary.scalar('loss', cross_entropy)
  with tf.name_scope('accuracy'):
    accuracy = tf.compat.v1.metrics.accuracy(labels=labels, predictions=predicted_logit, name='acc')
    tf.summary.scalar('accuracy', accuracy[1])



  # Create a hook to print acc, loss & global step every 100 iter.   
  eval_hook_list= []
  eval_tensors_log = {'accuracy': accuracy[1],'loss': cross_entropy,'global_step': global_step}
  eval_hook_list.append(tf.compat.v1.train.LoggingTensorHook(tensors=eval_tensors_log, every_n_iter=100))
  eval_summary_hook = tf.compat.v1.train.SummarySaverHook(10,output_dir='step3/data/evaluation',summary_op=tf.compat.v1.summary.scalar("accuracy",accuracy[1]))
  eval_hook_list.append(eval_summary_hook)
        
        
  #EVAL
  if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode,loss=cross_entropy,eval_metric_ops={'accuracy/accuracy': accuracy},evaluation_hooks=eval_hook_list)
      
    
  # Create a SGD optimizer 
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=_LEARNING_RATE)
  train_op = optimizer.minimize(cross_entropy,global_step=global_step)
    
  # Create a hook to print acc, loss & global step every 100 iter.   
  train_hook_list= []
  train_tensors_log = {'accuracy': accuracy[1],'loss': cross_entropy,'global_step': global_step}
  train_hook_list.append(tf.compat.v1.train.LoggingTensorHook(tensors=train_tensors_log, every_n_iter=100))
  summary_hook = tf.compat.v1.train.SummarySaverHook(10,output_dir='step3/data/training',summary_op=tf.compat.v1.summary.scalar("accuracy",accuracy[1]))
  train_hook_list.append(summary_hook)
    
  #TRAIN
  if mode == tf.estimator.ModeKeys.TRAIN:
    return tf.estimator.EstimatorSpec(mode=mode,loss=cross_entropy,train_op=train_op,training_hooks=train_hook_list)

 

In [ ]:
classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='step3')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'step3', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7febf5647d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 938: step3/model.ckpt-938


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 938: step3/model.ckpt-938


INFO:tensorflow:Loss for final step: 0.09473019.


INFO:tensorflow:Loss for final step: 0.09473019.


({'accuracy/accuracy': 0.94125, 'global_step': 938, 'loss': 0.205144}, [])